In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
import tensorflow_models as tfm

from official.vision.configs import maskrcnn as exp_cfg
from official.core import config_definitions as cfg
from official.core import exp_factory
from official.vision.tasks import maskrcnn
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder
from official.vision.serving import export_saved_model_lib
from official.vision.configs import backbones as backbones_cfg
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import os
import numpy as np
from official.vision.utils.object_detection import visualization_utils
import time
from custom_utils import send_pushover_notification, intercept_stdout
from functools import partial
from tfm_configs import *
import re
import tensorflow_datasets as tfds

In [3]:
IMAGE_SIZE = (640, 640)
BATCH_SIZE = 4
TFDS_NAME = 'leaf_instance_dataset'
INPUT_PATH = "/home/stefan.steinheber/tensorflow_datasets/leaf_instance_dataset/1.0.0/"
MODEL = "retinanet_resnet_fpn"
MODEL_DIR = "out/" + MODEL

In [4]:
exp_config = retinanet_resnet_fpn(INPUT_PATH)

In [5]:
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices('GPU')]

if len(logical_device_names) == 0:
    logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

distribution_strategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])

print("Created distribution Strategy on Device", logical_device_names[0])

Created distribution Strategy on Device /device:GPU:0


2024-07-16 13:42:03.010026: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45507 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:06:00.0, compute capability: 8.6


In [6]:
def show_batch(raw_records):
    tf_ex_decoder = TfExampleDecoder(include_mask=True)
    plt.figure(figsize=(20, 20))
    use_normalized_coordinates=True
    min_score_thresh = 0.30
    for i, serialized_example in enumerate(raw_records):
        plt.subplot(1, 3, i + 1)
        decoded_tensors = tf_ex_decoder.decode(serialized_example)
        image = decoded_tensors['image'].numpy().astype('uint8')
        scores = np.ones(shape=(len(decoded_tensors['groundtruth_boxes'])))
        visualization_utils.visualize_boxes_and_labels_on_image_array(
            image,
            decoded_tensors['groundtruth_boxes'].numpy(),
            decoded_tensors['groundtruth_classes'].numpy().astype('int'),
            scores,
            category_index={
                1: {
                    'id': 1,
                    'name': 'leaf',
                },
            },
            use_normalized_coordinates=use_normalized_coordinates,
            min_score_thresh=min_score_thresh,
            instance_masks=decoded_tensors['groundtruth_instance_masks'].numpy().astype('uint8'),
            line_thickness=4)

        plt.imshow(image)
        plt.axis("off")
        plt.title(f"Image-{i+1}")
    plt.show()
    #plt.savefig("out/fig.png")

In [7]:
with distribution_strategy.scope():
    task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=MODEL_DIR)

In [8]:
buffer_size = 100
num_of_examples = 3

train_tfrecords = tf.io.gfile.glob(exp_config.task.train_data.input_path)
raw_records = tf.data.TFRecordDataset(train_tfrecords).shuffle(buffer_size=buffer_size).take(num_of_examples)
show_batch(raw_records)

2024-07-16 13:42:07.208258: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
def send_notification(text):
    if "loss" not in text or 'eval' not in text:
        return
    step = re.search(r"step:.*?(\d+).*?\|", text)
    step = step.group(1)
    losses = re.findall(r"'(.*)_loss':.*?(\d+\.\d+)", text)
    losses.reverse()
    msg = f"Step #{step}:" + ' - '.join([f"{name} Loss: {value}" for name, value in losses])
    send_pushover_notification(msg, title="Training Losses", priority=-1)

In [ ]:
send_pushover_notification("Starting Training", "Tensorflow Models")

with intercept_stdout(send_notification):
    model, eval_logs = tfm.core.train_lib.run_experiment(
        distribution_strategy=distribution_strategy,
        task=task,
        mode='train_and_eval',
        params=exp_config,
        model_dir=MODEL_DIR,
        run_post_eval=True)

restoring or initializing model...
restored model from out/retinanet_resnet_fpn/ckpt-184200.
restored from checkpoint: out/retinanet_resnet_fpn/ckpt-184200
train | step:  184200 | training until step 185200...


2024-07-16 09:40:32.478372: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


train | step:  185200 | steps/sec:    4.0 | output: 
    {'box_loss': 0.0015649764,
     'cls_loss': 0.13380967,
     'learning_rate': 0.0012,
     'model_loss': 0.21205865,
     'total_loss': 0.33398414,
     'training_loss': 0.33398414}
saved checkpoint to out/retinanet_resnet_fpn/ckpt-185200.
 eval | step:  185200 | running 278 steps of evaluation...


2024-07-16 09:45:05.960424: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-07-16 09:45:05.960546: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-07-16 09:45:05.961395: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


 eval | step:  185200 | steps/sec:    5.5 | eval time:   50.7 sec | output: 
    {'box_loss': 0.0033559632,
     'cls_loss': 0.22008419,
     'model_loss': 0.38788235,
     'steps_per_second': 5.485444635236317,
     'total_loss': 0.50968564,
     'validation_loss': 0.50968564}
train | step:  185200 | training until step 186200...
train | step:  186200 | steps/sec:    3.8 | output: 
    {'box_loss': 0.0015796728,
     'cls_loss': 0.13456315,
     'learning_rate': 0.0012,
     'model_loss': 0.21354692,
     'total_loss': 0.33522737,
     'training_loss': 0.33522737}
saved checkpoint to out/retinanet_resnet_fpn/ckpt-186200.
 eval | step:  186200 | running 278 steps of evaluation...


...

 eval | step:  227200 | steps/sec:    6.8 | eval time:   41.0 sec | output: 
    {'box_loss': 0.003401585,
     'cls_loss': 0.23290111,
     'model_loss': 0.40298036,
     'steps_per_second': 6.77810452119328,
     'total_loss': 0.51526153,
     'validation_loss': 0.51526153}
train | step:  227200 | training until step 228200...
train | step:  228200 | steps/sec:    4.0 | output: 
    {'box_loss': 0.0013595847,
     'cls_loss': 0.11926267,
     'learning_rate': 0.0012,
     'model_loss': 0.18724187,
     'total_loss': 0.29941678,
     'training_loss': 0.29941678}
saved checkpoint to out/retinanet_resnet_fpn/ckpt-228200.
 eval | step:  228200 | running 278 steps of evaluation...


2024-07-16 12:47:35.697000: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-07-16 12:47:35.697119: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-07-16 12:47:35.697885: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


 eval | step:  228200 | steps/sec:    6.6 | eval time:   42.3 sec | output: 
    {'box_loss': 0.0034008278,
     'cls_loss': 0.22955768,
     'model_loss': 0.39959908,
     'steps_per_second': 6.5691786491398805,
     'total_loss': 0.51166886,
     'validation_loss': 0.51166886}
train | step:  228200 | training until step 229200...
train | step:  229200 | steps/sec:    3.9 | output: 
    {'box_loss': 0.0013595577,
     'cls_loss': 0.11950614,
     'learning_rate': 0.0012,
     'model_loss': 0.18748371,
     'total_loss': 0.29944828,
     'training_loss': 0.29944828}
saved checkpoint to out/retinanet_resnet_fpn/ckpt-229200.
 eval | step:  229200 | running 278 steps of evaluation...


2024-07-16 12:51:48.610366: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-07-16 12:51:48.610484: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-07-16 12:51:48.611221: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


 eval | step:  229200 | steps/sec:    6.8 | eval time:   41.1 sec | output: 
    {'box_loss': 0.0034227644,
     'cls_loss': 0.23410904,
     'model_loss': 0.40524724,
     'steps_per_second': 6.762283608466516,
     'total_loss': 0.5171072,
     'validation_loss': 0.5171072}
train | step:  229200 | training until step 230200...
train | step:  230200 | steps/sec:    4.0 | output: 
    {'box_loss': 0.0013263325,
     'cls_loss': 0.117364846,
     'learning_rate': 0.0012,
     'model_loss': 0.18368137,
     'total_loss': 0.295437,
     'training_loss': 0.295437}
saved checkpoint to out/retinanet_resnet_fpn/ckpt-230200.
 eval | step:  230200 | running 278 steps of evaluation...


In [9]:
send_pushover_notification("Finished Training", "Tensorflow Models")

export_saved_model_lib.export_inference_graph(
    input_type='image_tensor',
    batch_size=1,
    input_image_size=[IMAGE_SIZE[1], IMAGE_SIZE[0]],
    params=exp_config,
    checkpoint_path=tf.train.latest_checkpoint(MODEL_DIR),
    export_dir=f'{MODEL_DIR}/final')

INFO:tensorflow:Assets written to: out/retinanet_resnet_fpn/final/assets


INFO:tensorflow:Assets written to: out/retinanet_resnet_fpn/final/assets
